# Compute Spectral Centroid and Spectral Bandwidth for each plant and days

### Converting Plant Data into MIDI Control Curves

This notebook demonstrates how to convert plant data into MIDI control curves, generating a separate MIDI file for each day and each plant. The process involves extracting spectral features from the plant data and encoding these features as MIDI control change messages, which can be used in music production software like Ableton Live.

#### Steps Involved

1. **Importing Libraries**:
    - Import necessary libraries for audio processing, signal smoothing, numerical operations, MIDI file handling, and plotting.

2. **Function Definitions**:
    - **`compute_spectromorph`**: 
        - Processes raw plant data to compute spectral centroid and spectral bandwidth.
        - Smooths the spectral features using Savitzky-Golay filter.
        - Generates a time vector spanning 24 hours.
    - **`save_to_MIDIcc`**: 
        - Takes the time series data (time, smoothed spectral centroid, and smoothed spectral bandwidth).
        - Saves them as MIDI control change messages in a specified file.
        - Scales the control values to the range [0, 127] to fit MIDI standards.

3. **Processing and Saving Data**:
    - **`process_files`**: 
        - Iterates through the list of raw data files.
        - Processes each file to compute spectral features.
        - Saves the resulting control curves as MIDI files.
        - Names each MIDI file according to the processed plant data file.

In [1]:
import librosa
# import smoothing function
from scipy.signal import savgol_filter
import numpy as np
import mido
import numpy as np
import matplotlib.pyplot as plt
import os
from mido import MidiFile, MidiTrack, Message

def compute_spectromorph(data, sr=256, n_fft=10000, hop_length=1000):
    # remove nans
    data = data[~np.isnan(data)]
    # Compute spectral centroid and spectral_slope
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)
    spectral_centroid = spectral_centroid[0][10:-10] # Remove the edge effects
    sc_smoothed = savgol_filter(spectral_centroid, 1001, 3)
    #tonnetz = librosa.feature.tonnetz(y=data_, sr=10000,)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)
    spectral_bandwidth = spectral_bandwidth[0][10:-10] # Remove the edge effects
    sb_smoothed = savgol_filter(spectral_bandwidth, 1001, 3)
    # create time vector that matches the length of the spectral centroid on 24 hours
    time = np.linspace(0, 24, len(sc_smoothed))
    return time, sc_smoothed, sb_smoothed



def save_to_MIDIcc(time, sc_smoothed, sb_smoothed, filename):
    """
    Save the time series data to a MIDI file as control change messages.

    Args:
        time (np.ndarray): Array of time values.
        sc_smoothed (np.ndarray): Array of smoothed control values for control change 74.
        sb_smoothed (np.ndarray): Array of smoothed control values for control change 71.
        filename (str): The name of the MIDI file to save.
    """
    if not isinstance(time, np.ndarray) or not isinstance(sc_smoothed, np.ndarray) or not isinstance(sb_smoothed, np.ndarray):
        raise ValueError("Input arguments 'time', 'sc_smoothed', and 'sb_smoothed' must be numpy arrays.")

    if not isinstance(filename, str):
        raise ValueError("The 'filename' must be a string.")

    if len(time) != len(sc_smoothed) or len(time) != len(sb_smoothed):
        raise ValueError("The length of 'time', 'sc_smoothed', and 'sb_smoothed' must be equal.")

    # scale the control values to the range [0, 127]
    sc_smoothed = np.interp(sc_smoothed, (sc_smoothed.min(), sc_smoothed.max()), (0, 127))
    sb_smoothed = np.interp(sb_smoothed, (sb_smoothed.min(), sb_smoothed.max()), (0, 127))
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Add control change messages
    for i in range(len(time)):
        track.append(Message('control_change', control=20, value=int(sc_smoothed[i]), time=int(time[i])))
        track.append(Message('control_change', control=21, value=int(sb_smoothed[i]), time=int(time[i])))

    mid.save(filename)
    print(f'Successfully saved MIDI file: {filename}')

# Example usage
def process_files(filenames, datafolder, outputfolder, n_fft=10000, hop_length=1000):
    for file_path in filenames:
        try:
            data = np.fromfile(datafolder + file_path, dtype=np.float32)
            time, sc_smoothed, sb_smoothed = compute_spectromorph(data, sr=256, n_fft=n_fft, hop_length=hop_length)
            output_file = f'{outputfolder}/SpectroMorph_control_{file_path}.mid'
            save_to_MIDIcc(time, sc_smoothed, sb_smoothed, output_file)
            print('Saved file:', output_file)
        except Exception as e:
            print(f'Error processing file {file_path}: {e}')

# Example call to process_files
# Make sure to replace 'filenames', 'datafolder', and 'outputfolder' with actual values
datafolder = '../plant_data/Hiver 2024/Fichier RAW/'
filenames = os.listdir(datafolder)
process_files(filenames, datafolder, '../Plant_Music/Spectromorph_curves/', n_fft=10000, hop_length=1000)


Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-27.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-27.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-28.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-28.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-29.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-02-29.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-03-01.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer pensylvanicum 2024-03-01.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//Spectr

c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=0
  return f(*args, **kwargs)


Error processing file Acer pensylvanicum 2024-03-05_2.raw: If mode is 'interp', window_length must be less than or equal to the size of x.
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-27..raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-27..raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-28.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-28.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-29.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-02-29.raw.mid
Successfully saved MIDI file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-03-01.raw.mid
Saved file: ../Plant_Music/Spectromorph_curves//SpectroMorph_control_Acer rubrum 2024-0

c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=77
  return f(*args, **kwargs)
c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=86
  return f(*args, **kwargs)
c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=5600
  return f(*args, **kwargs)
c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=5592
  return f(*args, **kwargs)
c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=10000 is too small for input signal of length=5576
  return f(*args, **kwargs)
c:\Users\Antoine\anaconda3\envs\goofi2\lib\site-packages\librosa\util\deco

Error processing file config 2024-02-27.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-02-28.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-02-29.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-03-01.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-03-02.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-03-03.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-03-04.txt: If mode is 'interp', window_length must be less than or equal to the size of x.
Error processing file config 2024-03-05_1.txt: If mode is 'interp', window_length must be less than or equal to the size of x

In [ ]:
datafolder = '../plant_data/Hiver 2024/Fichier RAW/'
filenames = os.listdir(datafolder)
file_paths = filenames[18:23]
print(file_paths)
n_fft = 10000
hop_length = 1000

sc_smoothed = []

for file_path in file_paths:
    data = np.fromfile(datafolder + file_path, dtype=np.float32)
    time, sc_smoothed_, sb_smoothed = compute_spectromorph(data, sr=256, n_fft=n_fft, hop_length=hop_length)
    sc_smoothed.append(sc_smoothed_)


In [ ]:
# plot the smoothed spectral centroid for each file different colors from Set2

colors = plt.cm.Set1(np.linspace(0, 1, len(sc_smoothed)))
# match the length of the time vector to the length of the spectral centroid
time = np.linspace(0, 24, len(sc_smoothed[0]))
# create smoothed version of the spectral centroid
sc_smoothed2 = [savgol_filter(sc, 3001, 3) for sc in sc_smoothed]
dates = ['27-02', '28-02', '29-02', '01-03', '02-03']
for i, sc in enumerate(sc_smoothed):
    time = np.linspace(0, 24, len(sc))
    plt.plot(time, sc, color=colors[i], linewidth=1, linestyle='--', alpha=0.5)
    plt.plot(time, sc_smoothed2[i], color=colors[i], linewidth=2, label=dates[i])
plt.xlabel('Time (hours)')
plt.ylabel('Spectral Centroid')
plt.title('Alnus - Spectral Centroid')
# add legend with dates from 27-02 to 02-03 (it is bisextile year)
# add vertical lines at 6.5 and 17.6 and put soft beige background between them
plt.axvline(6.5, color='k', linestyle='--', linewidth=1)
plt.axvline(17.6, color='k', linestyle='--', linewidth=1)
plt.fill_between([6.5, 17.6], 1.5, 4, color='yellow', alpha=0.1)
# write sunrise and sunset times at the bottom of the plot
plt.text(8.2, 2.1, 'Sunrise', horizontalalignment='center')
plt.text(16, 2.1, 'Sunset', horizontalalignment='center')
plt.ylim(2, 3.75)
plt.legend()

# save the plot
plt.savefig('Spectral_Centroid_Alnus4.png')
plt.show()